ADHD PATIENT PREPROCESSING

In [2]:
!pip install mne

   ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
   --------- ------------------------------ 1.8/7.4 MB 10.3 MB/s eta 0:00:01
   --------------------- ------------------ 3.9/7.4 MB 9.5 MB/s eta 0:00:01
   ------------------------------- -------- 5.8/7.4 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------- 7.4/7.4 MB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --------- ------------------------------ 1.8/8.1 MB 9.6 MB/s eta 0:00:01
   ------------------- -------------------- 3.9/8.1 MB 9.4 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.1 MB 9.3 MB/s eta 0:00:01
   ------------------------------------- -- 7.6/8.1 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.6 MB 10.5 MB/s eta 0:00:02
   ------------ -------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import mne

def preprocess_eeg(file_path, output_dir, patient_id):
    # Load the EEG data
    raw = mne.io.read_raw_brainvision(file_path, preload=True)
    
    # Select the last 50 seconds of the recording
    raw.crop(tmax=raw.times[-1] - 50)
    
    # Apply band-pass filtering (0.5 to 100 Hz)
    raw.filter(0.5, 100, fir_design='firwin')
    
    # Apply notch filter at 50 Hz
    raw.notch_filter(50, fir_design='firwin')
    
    # Set the montage (10-10 system)
    montage = mne.channels.make_standard_montage("standard_1010")
    raw.set_montage(montage)
    
    # Apply ICA for artifact removal 

    ica = mne.preprocessing.ICA(n_components=25, method='fastica', max_iter=500, random_state=42)
    
    # Fit ICA
    ica.fit(raw)
    
    # Automatically detect artifacts
    ecg_indices, _ = ica.find_bads_ecg(raw)
    eog_indices, _ = ica.find_bads_eog(raw)
    emg_indices, _ = ica.find_bads_muscle(raw)  # Detect EMG artifacts
    
    # Exclude artifacts and apply ICA
    ica.exclude = ecg_indices + eog_indices + emg_indices
    raw = ica.apply(raw)
    
    # Save preprocessed EEG data in original format
    output_vhdr = os.path.join(output_dir, f"{patient_id}-preprocessed.vhdr")
    output_eeg = os.path.join(output_dir, f"{patient_id}-preprocessed.eeg")
    output_mrk = os.path.join(output_dir, f"{patient_id}-preprocessed.vmrk")
    
    os.makedirs(output_dir, exist_ok=True)
    raw.export(output_vhdr, overwrite=True)
    
    return raw

def process_all_patients(data_folder, output_folder):
    all_raw = []
    
    for patient_folder in os.listdir(data_folder):
        patient_path = os.path.join(data_folder, patient_folder, "ses-1")
        
        # Find the .vhdr file
        vhdr_file = [f for f in os.listdir(patient_path) if f.endswith('.vhdr')]
        
        if vhdr_file:
            file_path = os.path.join(patient_path, vhdr_file[0])
            patient_id = patient_folder  # Use folder name as patient ID
            raw_data = preprocess_eeg(file_path, output_folder, patient_id)
            all_raw.append(raw_data)
    
    return all_raw

# Example usage
data_folder = "I:/adhd/adhd-patients"
output_folder = "I:/adhd/adhd-patients-preprocessed"
raw_list = process_all_patients(data_folder, output_folder)
